In [202]:
import os
import json
import time
import random
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent
from openpyxl.utils import get_column_letter

In [128]:
CDK = "3c51dd63-4127-4f0b-a9b9-42274e246195"
BASE_URL = "https://game.wangzhedingwei.com/api/v1/cdk/rank"
ua = UserAgent(use_cache_server=False)

In [129]:
game_area_dict = {
    "苹果微信": 1,
    "安卓微信": 2,
    "苹果QQ": 3,
    "安卓QQ": 4,
}

In [130]:
hero_list = requests.get(
        url="https://game.wangzhedingwei.com/api/v1/hero/list?keywords=&output=json",
        headers={'User-Agent': ua.random}
    ).json()["data"]["data"]

hero_list = pd.DataFrame(hero_list)[["id", "name", "photo_url"]].rename(columns={
    "id": "英雄ID",
    "name": "英雄名称",
    "photo_url": "英雄头像"
})

In [131]:
hero_ids = hero_list["英雄ID"].unique()

In [132]:
hero_rank_info = []
for platform in game_area_dict.keys():
    for index in tqdm(hero_ids, total=len(hero_ids), desc=platform):
        url = f"{BASE_URL}?cdk={CDK}&game_area={game_area_dict[platform]}&hero_id={index}"
        
        response_info = requests.get(url=url, headers={'User-Agent': ua.random}).json()["data"]

        for t in ["city", "discrict", "province"]:
            info = response_info[t]
            
            if info:
                game_area = info["game_area"]
                hero_id = info["wzry_hero_id"]
                rank_type = info["rank_type"]
                rank_info = json.loads(info["rank_info"])
                
                for rank_info in rank_info:
                    hero_rank_info.append({
                        "英雄ID": hero_id,
                        "游戏平台": game_area,
                        "榜单类型": rank_type,
                        "地区编码": rank_info["adcode"],
                        "排名地区": rank_info["address"],
                        "上榜战力": rank_info["rank"],
                    })

安卓QQ: 100%|██████████| 109/109 [00:24<00:00,  4.39it/s]


In [207]:
hero_rank = pd.DataFrame(hero_rank_info)
hero_rank["游戏平台"] = hero_rank["游戏平台"].map({v: k for k, v in game_area_dict.items()})
hero_rank["榜单类型"] = hero_rank["榜单类型"].map({1: "省榜", 2: "市榜", 3: "区榜"})
hero_rank["上榜战力"] = hero_rank["上榜战力"].astype("int")

hero_rank_list = hero_list.merge(hero_rank, on="英雄ID")
hero_rank_list = hero_rank_list.sort_values(["英雄ID", "上榜战力"]).reset_index(drop=True)

In [208]:
# result = hero_rank_list[['英雄ID', '英雄名称', '游戏平台', '榜单类型', '地区编码', '排名地区', '上榜战力', '英雄头像']]
result = hero_rank_list[['英雄名称', '游戏平台', '榜单类型', '排名地区', '上榜战力']]

In [203]:
def to_excel_auto_column_weight(df: pd.DataFrame, writer: pd.ExcelWriter, sheet_name):
    """DataFrame保存为excel并自动设置列宽"""
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    #  计算表头的字符宽度
    column_widths = (
        df.columns.to_series().apply(lambda x: len(x.encode('gbk'))).values
    )
    #  计算每列的最大字符宽度
    max_widths = (
        df.astype(str).applymap(lambda x: len(x.encode('gbk'))).agg(max).values
    )
    # 计算整体最大宽度
    widths = np.max([column_widths, max_widths], axis=0)
    # 设置列宽
    worksheet = writer.sheets[sheet_name]
    for i, width in enumerate(widths, 1):
        # openpyxl引擎设置字符宽度时会缩水0.5左右个字符，所以干脆+2使左右都空出一个字宽。
        worksheet.column_dimensions[get_column_letter(i)].width = width + 2

In [209]:
with pd.ExcelWriter(f'全英雄最低战力战区查询-{time.strftime("%Y-%m-%d", time.localtime())}.xlsx', engine="openpyxl") as writer:
    to_excel_auto_column_weight(result, writer, f'最低战力战区')

In [ ]:
# result.to_excel(f'全英雄最低战力战区查询-{time.strftime("%Y-%m-%d", time.localtime())}.xlsx', index=False)